# 1.c. Continuous reservoir simulations


So far, we have utilized NSGA-II provided by iRONS to generate the Pareto front for reservoir release schedules (1.a. Initial reservoir release optimization). Subsequently, we have selected a compromising release schedule from this Pareto front using eight different Multi-Criteria Decision-Making (MCDM) techniques (1.b. Selection of compromise solution from the Pareto front).

The aforementioned process needs to be conducted iteratively every month (for monthly decision-making) or every two months (for bimonthly decision-making) throughout the simulation period. The provided code facilitates this continuous simulation by utilizing the previously defined functions.



![continuous_simulation](util/images/continuous_simulation.jpg)

## 1. Import libraries

We start to importing the necessary libraries and tools (🚨 in order to run the code like in the box below, place the mouse pointer in the cell, then click on “run cell” button above or press shift + enter).

In [84]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from platypus import NSGAII, Problem, Real
import warnings
warnings.filterwarnings('ignore')
import datetime
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator
import calendar
from datetime import date
from dateutil.relativedelta import relativedelta
from irons.Software.read_data import read_csv_data
from irons.Software.day2week2month import day2week
from irons.Software.res_sys_sim import res_sys_sim
from sklearn.preprocessing import MinMaxScaler
path = os.getcwd()

## 2. Read functions for reservoir simulations

Here, we read the functions that we defined from 1.a (release schedule optimisation) and 1.b (selection of compromise release schedule from the Pareto front) for continuous simulations.

### 2.1 Functions for release optimisation

In [142]:
def reservoir_opt_deterministic(reservoir_name,month,year,leadtime,pop, itnum, scenario):
    
    input_path = path + '/data/' + scenario_list[scenario] + '/in/leadtime_' + str(leadtime) + '/' # folder containing the forecast weather data
    observation_path = path + '/data/' + scenario_list[1] + '/in/leadtime_' + str(leadtime) + '/' # folder containing the forecast weather data    
    output_path = path + '/data/' + scenario_list[scenario] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] + '/' # folder containing the forecast weather data

    # Read potential evapotranspiration forecasts
    e_fore_file = 'e_fore_' + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, e_fore_day = read_csv_data(input_path, e_fore_file)
    dates_fore, e_fore_ens, e_fore_cum_ens = day2week(dates_fore_day, e_fore_day)
    e_fore = np.zeros((dates_fore.shape[0],1))
    e_fore[:,0] = e_fore_ens.mean(1)   # Average of the forecast ensemble
    
    # Read potential evapotranspiration observation
    e_obs_file = 'e_fore_' + str(year) + '_' + str(month) + '.csv'
    dates_obs_day, e_obs_day = read_csv_data(observation_path, e_obs_file)
    dates_obs, e_obs_ens, e_obs_cum_ens = day2week(dates_obs_day,e_obs_day)
    e_obs = np.zeros((dates_obs.shape[0],1))
    e_obs[:,0] = e_obs_ens.mean(1)      # Average of the forecast ensemble

    # Read inflow forecasts
    I_fore_file = 'I_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, I_fore_day = read_csv_data(input_path, I_fore_file)
    dates_fore, I_fore_ens, I_fore_cum_ens = day2week(dates_fore_day,I_fore_day)
    I_fore = np.zeros((dates_fore.shape[0],1))
    I_fore[:,0] = I_fore_ens.mean(1)      # Average of the forecast ensemble

    # Read inflow observation
    I_obs_file = 'I_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_obs_day, I_obs_day = read_csv_data(observation_path, I_obs_file)
    dates_obs, I_obs_ens, I_obs_cum_ens = day2week(dates_obs_day,I_obs_day)
    I_obs = np.zeros((dates_obs.shape[0],1))
    I_obs[:,0] = I_obs_ens.mean(1)         # Average of the forecast ensemble

    # Read demand (fixed)
    d_fore_file = 'd_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, d_fore_day = read_csv_data(input_path, d_fore_file)
    dates_fore, d_fore_ens, d_fore_cum_ens = day2week(dates_fore_day,d_fore_day)
    d_fore = np.zeros((dates_fore.shape[0],1))
    d_fore[:,0] = d_fore_ens.mean(1)        # Average of the forecast ensemble
    

    # Read observation data and set initial storage
    obsdata = pd.read_csv(path + '/data/observed_data.csv',index_col = 'Date')
    obsdata.index = pd.to_datetime(obsdata.index)
    
    if year == start_year and month == start_month :
        ini_date = dates_fore[0] # the initial date corresponds to the first date of the weekly forecast
        s_ini = obsdata[obsdata.index == ini_date].iloc[0,0]    
    else:
        if scenario == 1:
            s_ini = S_all['S_pf'][len(S_all)-1]
        elif scenario == 2:
            s_ini = S_all['S_wc'][len(S_all)-1]
        elif scenario == 3:
            s_ini = S_all['S_d20'][len(S_all) - 1]
    
    M = 1                                   # Number of Ensemble (if M=1, deterministic)
    N = dates_fore.shape[0]                 # Number of forecast weeks
    
    # Define the target week for Storage Optimisation
    if month + leadtime <10:
        target_week = N
    elif month >=10:
        target_week = N
    else:
        target_week = N - 4*(month + leadtime - 10)
  
    # Release optimisation
    pop_size = pop   # population size
    num_iter = itnum # Number of iterations
    
    def auto_optim(vars):
        Qreg_s_D = np.zeros((N,1))
        # Decision vector: regulated release to meet the demand in D
        Qreg_s_D[:,0] = np.array(vars[0:N])
        Qreg = {'releases' : {'type'  : 'scheduling',
                              'input' : Qreg_s_D},
                'inflows'  : [],
                'rel_inf'  : []}
    
        # Reservoir system simulation
        Qenv, Qspill, Qreg_s_D, I_reg, s, E = res_sys_sim(I_fore, e_fore, s_ini, s_min, s_max,
                                                          env_min, d_fore, Qreg)
    
        # Objective functions
        SuDe = (np.maximum(d_fore-Qreg_s_D,0))**2   # supply deficit objective
        StDi = s_max - s                            # storage difference objective
        SSD = np.mean(SuDe[1:])                     # mean Squared Supply Deficit
        SVD = np.mean(StDi[target_week])            # Storage Volume Difference
    
        return [SSD, SVD]
    
    problem = Problem(N,2)                     # Problem(number of optimizer variables, num of objective functions)
    Real0 = Real(Qreg_s_D_min, Qreg_s_D_max)     # Range of values
    
    problem.types[:] = [Real0]*(N)
    problem.function = auto_optim
    
    algorithm_1 = NSGAII(problem,pop_size)
    algorithm_1.run(num_iter)
    
    sol_optim_1 = [algorithm_1.result[i].variables for i in range(pop_size)]
    
    rel_schedule = pd.DataFrame(sol_optim_1)               # array? csv? ???? ?? ???? ?? ??? ????? ?????
    rel_schedule = rel_schedule.transpose()
    rel_schedule['mean'] = rel_schedule.mean(numeric_only=True, axis=1)
    rel_schedule.to_csv(output_path + '[opt_rel_schedule]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                       + '_decision_' + str(decision_time) + 'm.csv')
    
    
    results_SSD_1 = np.array([algorithm_1.result[i].objectives[0] for i in range(pop_size)])
    results_SVD_1 = np.array([algorithm_1.result[i].objectives[1] for i in range(pop_size)])
        
    SSD = pd.DataFrame(results_SSD_1).rename(columns = {0:'SSD'})                      # array? csv? ???? ?? ???? ?? ??? ????? ?????
    SVD = pd.DataFrame(results_SVD_1).rename(columns = {0:'SVD'})                      # array? csv? ???? ?? ???? ?? ??? ????? ?????
    SSD_SVD = pd.concat([SSD, SVD], axis=1)
    SSD_SVD.to_csv(output_path + '[sim_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')
            
    temp = pd.DataFrame()
    
    for i in range(0,pop_size):
        Qreg_s_D_opt = np.reshape(sol_optim_1[i], (N,1))
        # Decision vector: regulated release to meet the demand in D
    
        Qreg_opt = {'releases' : {'type'  : 'scheduling',
                                'input' : Qreg_s_D_opt},
                'inflows'  : [],
                'rel_inf'  : []}
        
        Qenv, Qspill, Qreg_s_D_opt, I_reg, s, E = res_sys_sim(I_obs, e_obs, s_ini, s_min, s_max, 
                                                              env_min, d_fore, Qreg_opt)
        
        SuDe = (np.maximum(d_fore-Qreg_s_D_opt,0))**2   # supply deficit objective
        StDi = s_max - s                                # resource deficit objective
        SSD = np.mean(SuDe[1:])
        SVD = np.mean(StDi[target_week])
        
        output = pd.DataFrame(columns=['i', 'SSD', 'SVD'])    # array? csv? ???? ?? ???? ?? ??? ????? ?????
        output.loc[i] = [i, round(SSD, 3), round(SVD, 3)]
    
        temp = pd.concat([temp, output], axis=0)
    temp.set_index('i', inplace=True)
    temp.to_csv(output_path + '[act_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')

    # Visualsing Actual Pareto-front -------------------------------------------------------------------------------------------------------------------------------------
    plt.figure(figsize = (6,5))
    plt.grid(True, axis='y', linestyle=':')
    plt.grid(False, axis='x', linestyle=':')
    
    plt.scatter('SSD', 'SVD', data=temp, color='blue', alpha=0.21, s=50)
    plt.title(reservoir_name + ' ('+ str(month) + '/' + str(year) + ')', x=0.5, y=0.91)
    plt.xlabel('(SSD) Squared Supply Defict [MCM^2]')
    plt.ylabel('(SVD) Storage Volume Difference [MCM]')
    plt.savefig(output_path + '[act_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                        + '_decision_' + str(decision_time) + 'm.png', dpi=150, bbox_inches='tight')
    plt.close('all')
    
    # Operation data management -------------------------------------------------------------------------------------------------------------------------------------
    df = pd.read_csv(output_path + '[opt_rel_schedule]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                       + '_decision_' + str(decision_time) + 'm.csv', index_col = [0])

    Iobs = df.copy()
    for i in range(0,pop_size):
        Iobs[str(i)] = I_obs
        Iobs['mean'] = I_obs
    
    Demand = df.copy()
    for i in range(0,pop_size):
        Demand[str(i)] = d_fore        
        Demand['mean'] = d_fore
        
    Sini = df.copy().drop(df.index[1:])
    for i in range(0,pop_size):
        Sini[str(i)] = s_ini
        Sini['mean'] = s_ini
        
    df_merge = pd.concat([df, Iobs, Demand, Sini], axis=0).reset_index(drop=True)
    
    for i in range(0,len(df)): #storage volume calculation
        df_merge.loc[len(df)*3+1+i] = np.minimum(df_merge.loc[len(df)*3+i] - df_merge.loc[i] + df_merge.loc[len(df)+i], s_max)
        
    for i in range(0,len(df)+1): #storage volume difference calculation
        df_merge.loc[len(df)*4+1+i] = s_max - df_merge.loc[len(df)*3+i]
        
    for i in range(0,len(df)): #supply deficit calculation
        df_merge.loc[len(df)*5+2+i] =  np.maximum(df_merge.loc[len(df)*2+i] - df_merge.loc[i],0)
        
    df_merge.loc[len(df)*6+2] = df_merge.iloc[len(df)*5+2:len(df)*6+2].sum() # supply deficit sum calculation
    df_merge.loc[len(df)*6+3] = df_merge.iloc[len(df)*4+2:len(df)*5+2].mean() # mean SVD
    df_merge.loc[len(df)*6+4] = df_merge.iloc[len(df)*5+3:len(df)*6+2].pow(2).mean() # mean SSD
    df_merge.loc[len(df)*6+5] = df_merge.iloc[len(df)*5+3:len(df)*6+2].pow(2).sum() # mean SSD
    
    df_merge['remark'] = np.NaN
    for i in range(0, len(df)):
        df_merge['remark'].loc[i] = 'Release'
        df_merge['remark'].loc[len(df) + i] = 'I_obs'
        df_merge['remark'].loc[len(df)*2 + i] = 'Demand'
        df_merge['remark'].loc[len(df)*3] = 'Initial Sotrage'
        df_merge['remark'].loc[len(df)*3 + 1 + i] = 'Storage'
    for i in range(0, len(df)+1):
        df_merge['remark'].loc[len(df)*4 + 1 + i] = 'Volume Difference'
        df_merge['remark'].loc[len(df)*5 + 1 + i+1] = 'Supply Deficit'
        df_merge['remark'].loc[len(df)*6 + 1+1] = 'Sum of Supply Deficit'
        df_merge['remark'].loc[len(df)*6 + 2+1] = 'mean SVD'
        df_merge['remark'].loc[len(df)*6 + 3+1] = 'mean SSD'
        df_merge['remark'].loc[len(df)*6 + 4+1] = 'sum SSD'
    df_merge = df_merge.set_index('remark')
    
    df_merge.to_csv(output_path + '[Res_operation]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')

In [143]:
def reservoir_opt_probabilistic(reservoir_name,month,year,leadtime,pop, itnum, scenario):
    
    input_path = path + '/data/' + scenario_list[scenario] + '/in/leadtime_' + str(leadtime) + '/' # folder containing the forecast weather data
    observation_path = path + '/data/' + scenario_list[1] + '/in/leadtime_' + str(leadtime) + '/' # folder containing the forecast weather data    
    output_path = path + '/data/' + scenario_list[scenario] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] + '/' # folder containing the forecast weather data
    
    # Read potential evapotranspiration forecasts
    e_fore_file = 'e_fore_' + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, e_fore_day = read_csv_data(input_path, e_fore_file)
    dates_fore, e_fore_ens, e_fore_cum_ens = day2week(dates_fore_day, e_fore_day)
    e_fore = np.zeros((dates_fore.shape[0],1))
    e_fore[:,0] = e_fore_ens.mean(1)   # Average of the forecast ensemble
    
    # Read potential evapotranspiration observation
    e_obs_file = 'e_fore_' + str(year) + '_' + str(month) + '.csv'
    dates_obs_day, e_obs_day = read_csv_data(observation_path, e_obs_file)
    dates_obs, e_obs_ens, e_obs_cum_ens = day2week(dates_obs_day,e_obs_day)
    e_obs = np.zeros((dates_obs.shape[0],1))
    e_obs[:,0] = e_obs_ens.mean(1)      # Average of the forecast ensemble

    # Read inflow forecasts
    I_fore_file = 'I_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, I_fore_day = read_csv_data(input_path, I_fore_file)
    dates_fore, I_fore_ens, I_fore_cum_ens = day2week(dates_fore_day,I_fore_day)
    I_fore = np.zeros((dates_fore.shape[0],1))
    I_fore[:,0] = I_fore_ens.mean(1)      # Average of the forecast ensemble

    # Read inflow observation
    I_obs_file = 'I_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_obs_day, I_obs_day = read_csv_data(observation_path, I_obs_file)
    dates_obs, I_obs_ens, I_obs_cum_ens = day2week(dates_obs_day,I_obs_day)
    I_obs = np.zeros((dates_obs.shape[0],1))
    I_obs[:,0] = I_obs_ens.mean(1)         # Average of the forecast ensemble

    # Read demand (fixed)
    d_fore_file = 'd_fore_'  + str(year) + '_' + str(month) + '.csv'
    dates_fore_day, d_fore_day = read_csv_data(input_path, d_fore_file)
    dates_fore, d_fore_ens, d_fore_cum_ens = day2week(dates_fore_day,d_fore_day)
    d_fore = np.zeros((dates_fore.shape[0],1))
    d_fore[:,0] = d_fore_ens.mean(1)        # Average of the forecast ensemble

    # Read observation data and set initial storage
    obsdata = pd.read_csv(path + '/data/observed_data.csv',index_col = 'Date')
    obsdata.index = pd.to_datetime(obsdata.index)
        
    if year == start_year and month == start_month :
        ini_date = dates_fore[0] # the initial date corresponds to the first date of the weekly forecast
        s_ini = obsdata[obsdata.index == ini_date].iloc[0,0]    
    else:
        if scenario == 1:
            s_ini = S_all['S_pf'][len(S_all)-1]
        elif scenario == 2:
            s_ini = S_all['S_wc'][len(S_all)-1]
        elif scenario == 3:
            s_ini = S_all['S_d20'][len(S_all) - 1]
        elif scenario == 4:
            s_ini = S_all['S_esp'][len(S_all)-1]
        elif scenario == 5:
            s_ini = S_all['S_sffs'][len(S_all)-1]   
    
    M = I_fore_ens.shape[1] # M = Number of Ensemble or scenario
    N = dates_fore.shape[0]                 # Number of forecast weeks
    
    # Define the target week for Storage Optimisation
    if month + leadtime <10:
        target_week = N
    elif month >=10:
        target_week = N
    else:
        target_week = N - 4*(month + leadtime - 10)
    
    
    # Read observation data
    obsdata = pd.read_csv(path + '/data/observed_data.csv',index_col = 'Date')
    obsdata.index = pd.to_datetime(obsdata.index)
    
    # Release optimisation
    pop_size = pop   # population size
    num_iter = itnum # Number of iterations
    
    SSD = 0
    SVD = 0
                
    def auto_optim(vars):
        Qreg_S_D = np.array(vars[0:N]).reshape(N,1)
        # Decision vector: regulated release to meet the demand in D
        Qreg = {'releases' : {'type'  : 'scheduling',
                                'input' : Qreg_S_D},
                'inflows'  : [],
                'rel_inf'  : []}
                
        # Reservoir system simulation
        Qenv, Qspill, Qreg_S_D, I_reg, s, E = res_sys_sim(I_fore_ens,e_fore_ens, s_ini,s_min,s_max, 
                                                          env_min,d_fore, Qreg)
                
        # Objective functions
        SuDe = (np.maximum(d_fore_ens-Qreg_S_D,0))**2   # supply deficit objective
        StDi = s_max - s                            # resource deficit objective
        SSD = np.mean(np.mean(SuDe[1:,:],axis = 0))
        SVD = np.mean(np.mean(StDi[target_week,:],axis = 0))
                
        return [SSD, SVD]

    problem = Problem(N,2)                     # Problem(number of optimizer variables, num of objective functions)
    Real0 = Real(Qreg_s_D_min, Qreg_s_D_max)     # Range of values
    
    problem.types[:] = [Real0]*(N)
    problem.function = auto_optim
    
    algorithm_2 = NSGAII(problem,pop_size)
    algorithm_2.run(num_iter)
    
    sol_optim_2 = np.array([algorithm_2.result[i].variables for i in range(pop_size)])
    
    rel_schedule = pd.DataFrame(sol_optim_2)               # array? csv? ???? ?? ???? ?? ??? ????? ?????
    rel_schedule = rel_schedule.transpose()
    rel_schedule['mean'] = rel_schedule.mean(numeric_only=True, axis=1)
    rel_schedule.to_csv(output_path + '[opt_rel_schedule]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                       + '_decision_' + str(decision_time) + 'm.csv')
    
    
    results_SSD_2 = np.array([algorithm_2.result[i].objectives[0] for i in range(pop_size)])
    results_SVD_2 = np.array([algorithm_2.result[i].objectives[1] for i in range(pop_size)])
        
    SSD = pd.DataFrame(results_SSD_2).rename(columns = {0:'SSD'})                      # array? csv? ???? ?? ???? ?? ??? ????? ?????
    SVD = pd.DataFrame(results_SVD_2).rename(columns = {0:'SVD'})                      # array? csv? ???? ?? ???? ?? ??? ????? ?????
    SSD_SVD = pd.concat([SSD, SVD], axis=1)
    SSD_SVD.to_csv(output_path + '[sim_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')
            
    temp = pd.DataFrame()
    
    for i in range(0,pop_size):
        Qreg_s_D_opt = np.reshape(sol_optim_2[i], (N,1))
        # Decision vector: regulated release to meet the demand in D
    
        Qreg_opt = {'releases' : {'type'  : 'scheduling',
                                'input' : Qreg_s_D_opt},
                'inflows'  : [],
                'rel_inf'  : []}
        Qenv, Qspill, Qreg_s_D_opt, I_reg, s, E = res_sys_sim(I_obs, e_obs, s_ini, s_min, s_max, 
                                                              env_min, d_fore, Qreg_opt)
        
        SuDe = (np.maximum(d_fore-Qreg_s_D_opt,0))**2   # supply deficit objective
        StDi = s_max - s                                # resource deficit objective
        SSD = np.mean(SuDe[1:])
        SVD = np.mean(StDi[target_week])
        
        output = pd.DataFrame(columns=['i', 'SSD', 'SVD'])    # array? csv? ???? ?? ???? ?? ??? ????? ?????
        output.loc[i] = [i, round(SSD, 3), round(SVD, 3)]
    
        temp = pd.concat([temp, output], axis=0)
    temp.set_index('i', inplace=True)
    temp.to_csv(output_path + '[act_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')

    # Visualsing Actual Pareto-front -------------------------------------------------------------------------------------------------------------------------------------
    plt.figure(figsize = (6,5))
    plt.grid(True, axis='y', linestyle=':')
    plt.grid(False, axis='x', linestyle=':')
    
    plt.scatter('SSD', 'SVD', data=temp, color='blue', alpha=0.21, s=50)
    plt.title(reservoir_name + ' ('+ str(month) + '/' + str(year) + ')', x=0.5, y=0.91)
    plt.xlabel('(SSD) Squared Supply Defict [MCM^2]')
    plt.ylabel('(SVD) Storage Volume Difference [MCM]')
    plt.savefig(output_path + '[act_pareto]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                        + '_decision_' + str(decision_time) + 'm.png', dpi=150, bbox_inches='tight')
    plt.close('all')
    
    # Operation data management -------------------------------------------------------------------------------------------------------------------------------------
    df = pd.read_csv(output_path + '[opt_rel_schedule]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                       + '_decision_' + str(decision_time) + 'm.csv', index_col = [0])

    Iobs = df.copy()
    for i in range(0,pop_size):
        Iobs[str(i)] = I_obs_ens
        Iobs['mean'] = I_obs_ens
    
    Demand = df.copy()
    for i in range(0,pop_size):
        Demand[str(i)] = d_fore        
        Demand['mean'] = d_fore
        
    Sini = df.copy().drop(df.index[1:])
    for i in range(0,pop_size):
        Sini[str(i)] = s_ini
        Sini['mean'] = s_ini
        
    df_merge = pd.concat([df, Iobs, Demand, Sini], axis=0).reset_index(drop=True)
    
    for i in range(0,len(df)): #storage volume calculation
        df_merge.loc[len(df)*3+1+i] = np.minimum(df_merge.loc[len(df)*3+i] - df_merge.loc[i] + df_merge.loc[len(df)+i], s_max)
        
    for i in range(0,len(df)+1): #storage volume difference calculation
        df_merge.loc[len(df)*4+1+i] = s_max - df_merge.loc[len(df)*3+i]
        
    for i in range(0,len(df)): #supply deficit calculation
        df_merge.loc[len(df)*5+2+i] =  np.maximum(df_merge.loc[len(df)*2+i] - df_merge.loc[i],0)
        
    df_merge.loc[len(df)*6+2] = df_merge.iloc[len(df)*5+2:len(df)*6+2].sum() # supply deficit sum calculation
    df_merge.loc[len(df)*6+3] = df_merge.iloc[len(df)*4+2:len(df)*5+2].mean() # mean SVD
    df_merge.loc[len(df)*6+4] = df_merge.iloc[len(df)*5+3:len(df)*6+2].pow(2).mean() # mean SSD
    df_merge.loc[len(df)*6+5] = df_merge.iloc[len(df)*5+3:len(df)*6+2].pow(2).sum() # mean SSD
    
    df_merge['remark'] = np.NaN
    for i in range(0, len(df)):
        df_merge['remark'].loc[i] = 'Release'
        df_merge['remark'].loc[len(df) + i] = 'I_obs'
        df_merge['remark'].loc[len(df)*2 + i] = 'Demand'
        df_merge['remark'].loc[len(df)*3] = 'Initial Sotrage'
        df_merge['remark'].loc[len(df)*3 + 1 + i] = 'Storage'
    for i in range(0, len(df)+1):
        df_merge['remark'].loc[len(df)*4 + 1 + i] = 'Volume Difference'
        df_merge['remark'].loc[len(df)*5 + 1 + i+1] = 'Supply Deficit'
        df_merge['remark'].loc[len(df)*6 + 1+1] = 'Sum of Supply Deficit'
        df_merge['remark'].loc[len(df)*6 + 2+1] = 'mean SVD'
        df_merge['remark'].loc[len(df)*6 + 3+1] = 'mean SSD'
        df_merge['remark'].loc[len(df)*6 + 4+1] = 'sum SSD'
    df_merge = df_merge.set_index('remark')
    
    df_merge.to_csv(output_path + '[Res_operation]' + reservoir_name + '_' + str(year) + '_' + str(month) 
                   + '_decision_' + str(decision_time) + 'm.csv')

### 2.2 Functions for MCDM

In [144]:
def MCDM_SAW(MCDM, year, month, reservoir_name, decision_time, leadtime):
    
    # Define file paths for different flow scenarios/forecasts
    path_pf = path + '/data/' + scenario_list[1] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Perfect forecast
    path_wc = path + '/data/' + scenario_list[2] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Worst-case
    path_d20 = path + '/data/' + scenario_list[3] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # 20-year return period drought
    path_esp = path + '/data/' + scenario_list[4] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]  # ESP (Ensemble Streamflow Prediction)
    path_sffs = path + '/data/' + scenario_list[5] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # SFFs (Synthetic Flow Forecasts)

    # Read simulated Pareto front data for each flow scenario/forecast
    sim_pf = pd.read_csv(path_pf + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_wc = pd.read_csv(path_wc + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_d20 = pd.read_csv(path_d20 + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_esp = pd.read_csv(path_esp + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_sffs = pd.read_csv(path_sffs + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)

    # Normalization of the simulated Pareto front data
    scaler = MinMaxScaler()
    sim_all = pd.concat([sim_pf, sim_wc, sim_d20, sim_esp, sim_sffs], axis=0)  # Combine data from all scenarios
    nor_all = pd.DataFrame(scaler.fit(sim_all).transform(sim_all)).rename({0: 'SSD', 1: 'SVD'}, axis=1)  # Normalize data
    nor_all['number'] = nor_all.index  # Add index column for identification
    nor_all['remark'] = 'Perfect forecast'  # Default remark for Perfect forecast scenario
    
    # Adjust remarks and indices for different scenarios
    for i in range(len(nor_all)):
        if (nor_all['number'][i] >= 100) and (nor_all['number'][i] < 200):
            nor_all['remark'][i] = 'Worst-case'
            nor_all['number'][i] = nor_all['number'][i] - 100
        elif (nor_all['number'][i] >= 200) and (nor_all['number'][i] < 300):
            nor_all['remark'][i] = '20-year drought'
            nor_all['number'][i] = nor_all['number'][i] - 200
        elif (nor_all['number'][i] >= 300) and (nor_all['number'][i] < 400):
            nor_all['remark'][i] = 'ESP'
            nor_all['number'][i] = nor_all['number'][i] - 300
        elif (nor_all['number'][i] >= 400) and (nor_all['number'][i] < 500):
            nor_all['remark'][i] = 'SFFs'
            nor_all['number'][i] = nor_all['number'][i] - 400
    
    # Assign weights based on the selected MCDM method
    if MCDM == 1:  # Balanced
        weight = weight_bal
    elif MCDM == 2:  # Supply prioritized
        weight = weight_sup
    elif MCDM == 3:  # Storage prioritized
        weight = weight_sto

    # Calculate scores and find the optimal number for each scenario based on SAW methods
    nor_pf = nor_all[(nor_all.remark == 'Perfect forecast')]
    nor_pf['score'] = abs(nor_pf['SSD'] * (1 - weight) + nor_pf['SVD'] * weight)
    opt_num_pf = nor_pf[(nor_pf['score'] == nor_pf['score'].min())].index[0]

    nor_wc = nor_all[(nor_all.remark == 'Worst-case')]
    nor_wc['score'] = abs(nor_wc['SSD'] * (1 - weight) + nor_wc['SVD'] * weight)
    opt_num_wc = nor_wc[(nor_wc['score'] == nor_wc['score'].min())].index[0]
    
    nor_d20 = nor_all[(nor_all.remark == '20-year drought')]
    nor_d20['score'] = abs(nor_d20['SSD'] * weight + nor_d20['SVD'] * (1 - weight))
    opt_num_d20 = nor_d20[(nor_d20['score'] == nor_d20['score'].min())].index[0]  # Find optimal solution        

    nor_esp = nor_all[(nor_all.remark == 'ESP')]
    nor_esp['score'] = abs(nor_esp['SSD'] * (1 - weight) + nor_esp['SVD'] * weight)
    opt_num_esp = nor_esp[(nor_esp['score'] == nor_esp['score'].min())].index[0]

    nor_sffs = nor_all[(nor_all.remark == 'SFFs')]
    nor_sffs['score'] = abs(nor_sffs['SSD'] * (1 - weight) + nor_sffs['SVD'] * weight)
    opt_num_sffs = nor_sffs[(nor_sffs['score'] == nor_sffs['score'].min())].index[0]

    # Load and save reservoir operation data for each scenario using the optimal numbers
    pf = pd.read_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    pf[str(opt_num_pf)].to_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_pf = pd.DataFrame(pf[str(opt_num_pf)].loc['Storage'])[:].rename(columns={str(opt_num_pf): 'S_pf'})[:decision_time * 4 + 1]

    wc = pd.read_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    wc[str(opt_num_wc)].to_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_wc = pd.DataFrame(wc[str(opt_num_wc)].loc['Storage'])[:].rename(columns={str(opt_num_wc): 'S_wc'})[:decision_time * 4 + 1]

    d20 = pd.read_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    d20[str(opt_num_d20)].to_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_d20 = pd.DataFrame(d20[str(opt_num_d20)].loc['Storage'])[:].rename(columns={str(opt_num_d20): 'S_d20'})[:decision_time * 4 + 1]    
        
    esp = pd.read_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    esp[str(opt_num_esp)].to_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_esp = pd.DataFrame(esp[str(opt_num_esp)].loc['Storage'])[:].rename(columns={str(opt_num_esp): 'S_esp'})[:decision_time * 4 + 1]

    sffs = pd.read_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sffs[str(opt_num_sffs)].to_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_sffs = pd.DataFrame(sffs[str(opt_num_sffs)].loc['Storage'])[:].rename(columns={str(opt_num_sffs): 'S_sffs'})[:decision_time * 4 + 1]

    # Combine storage data from all scenarios into a single DataFrame and return
    S_all = pd.concat([S_pf, S_wc, S_d20, S_esp, S_sffs], axis=1).reset_index(drop=True)
    
    return S_all

In [167]:
# sub function for Simple Selective method
def get_weight_ss(S_value):
    weight_mapping = {                                    # assign methods depending on predefined storage range
        (0, 2700): (1-weight_sto, weight_sto),            # Storage-prioritized 
        (2700, 3300): (1-weight_bal, weight_bal),           # Balanced
        (3300, float('inf')): (1-weight_sup, weight_sup)  # Supply-prioritized
    }
    for range_tuple, weights in weight_mapping.items():
        if range_tuple[0] < S_value <= range_tuple[1]:
            return weights
    return (0.5, 0.5)  # 기본값 (S값이 주어진 범위에 없는 경우)

# sub function for Multi-weighting method
def get_weight_mw(S_value):
    weight_mapping = {                      # assign weights depending on predefined storage range
        (0, 2500): (0.3, 0.7),          
        (2500, 2750): (0.4, 0.6),
        (2750, 3000): (0.5, 0.5),
        (3000, 3250): (0.6, 0.4),
        (3250, 3500): (0.7, 0.3),
        (3500, float('inf')): (0.8, 0.2)
    }
    for range_tuple, weights in weight_mapping.items():
        if range_tuple[0] < S_value <= range_tuple[1]:
            return weights
    return (0.5, 0.5)  # 기본값 (S값이 주어진 범위에 없는 경우)

# Main function for Variable Weighting methods
def MCDM_VW(MCDM, year, month, reservoir_name, decision_time, leadtime, scenario):
    # Define file paths for different flow scenarios/forecasts
    path_pf = path + '/data/' + scenario_list[1] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Perfect forecast
    path_wc = path + '/data/' + scenario_list[2] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Worst-case
    path_d20 = path + '/data/' + scenario_list[3] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # 20-year return period drought
    path_esp = path + '/data/' + scenario_list[4] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]  # ESP (Ensemble Streamflow Prediction)
    path_sffs = path + '/data/' + scenario_list[5] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # SFFs (Synthetic Flow Forecasts)

    # Read simulated Pareto front data for each flow scenario/forecast
    sim_pf = pd.read_csv(path_pf + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_wc = pd.read_csv(path_wc + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_d20 = pd.read_csv(path_d20 + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_esp = pd.read_csv(path_esp + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_sffs = pd.read_csv(path_sffs + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    
    # Normalization of the simulated Pareto front data
    scaler = MinMaxScaler()
    sim_all = pd.concat([sim_pf, sim_wc, sim_d20, sim_esp, sim_sffs], axis=0)  # Combine data from all scenarios
    nor_all = pd.DataFrame(scaler.fit(sim_all).transform(sim_all)).rename({0: 'SSD', 1: 'SVD'}, axis=1)  # Normalize data
    nor_all['number'] = nor_all.index  # Add index column for identification
    nor_all['remark'] = 'Perfect forecast'  # Default remark for Perfect forecast scenario
    
    for i in range(len(nor_all)):
        if (nor_all['number'][i] >= 100) and (nor_all['number'][i] < 200):
            nor_all['remark'][i] = 'Worst-case'
            nor_all['number'][i] = nor_all['number'][i] - 100
        elif (nor_all['number'][i] >= 200) and (nor_all['number'][i] < 300):
            nor_all['remark'][i] = '20-year drought'
            nor_all['number'][i] = nor_all['number'][i] - 200
        elif (nor_all['number'][i] >= 300) and (nor_all['number'][i] < 400):
            nor_all['remark'][i] = 'ESP'
            nor_all['number'][i] = nor_all['number'][i] - 300
        elif (nor_all['number'][i] >= 400) and (nor_all['number'][i] < 500):
            nor_all['remark'][i] = 'SFFs'
            nor_all['number'][i] = nor_all['number'][i] - 400
    
    if MCDM == 4:
        w_ssd, w_svd = get_weight_ss(S_value)
    elif MCDM == 5:
        w_ssd, w_svd = get_weight_mw(S_value)
    else:
        drop

    # Apply the scoring function to each scenario
    nor_pf = nor_all[nor_all.remark == 'Perfect forecast']
    nor_pf['score'] = abs(nor_pf['SSD'] * w_ssd + nor_pf['SVD'] * w_svd)
    opt_num_pf = nor_pf[(nor_pf['score'] == nor_pf['score'].min())].index[0]

    nor_wc = nor_all[nor_all.remark == 'Worst-case']
    nor_wc['score'] = abs(nor_wc['SSD'] * w_ssd + nor_wc['SVD'] * w_svd)
    opt_num_wc = nor_wc[(nor_wc['score'] == nor_wc['score'].min())].index[0]

    nor_d20 = nor_all[nor_all.remark == '20-year drought']
    nor_d20['score'] = abs(nor_d20['SSD'] * w_ssd + nor_d20['SVD'] * w_svd)
    opt_num_d20 = nor_d20[(nor_d20['score'] == nor_d20['score'].min())].index[0]

    nor_esp = nor_all[nor_all.remark == 'ESP']
    nor_esp['score'] = abs(nor_esp['SSD'] * w_ssd + nor_esp['SVD'] * w_svd)
    opt_num_esp = nor_esp[(nor_esp['score'] == nor_esp['score'].min())].index[0]

    nor_sffs = nor_all[nor_all.remark == 'SFFs']
    nor_sffs['score'] = abs(nor_sffs['SSD'] * w_ssd + nor_sffs['SVD'] * w_svd)
    opt_num_sffs = nor_sffs[(nor_sffs['score'] == nor_sffs['score'].min())].index[0]
    
    # Load and save reservoir operation data for each scenario using the optimal numbers
    pf = pd.read_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    pf[str(opt_num_pf)].to_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_pf = pd.DataFrame(pf[str(opt_num_pf)].loc['Storage'])[:].rename(columns={str(opt_num_pf): 'S_pf'})[:decision_time * 4 + 1]

    wc = pd.read_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    wc[str(opt_num_wc)].to_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_wc = pd.DataFrame(wc[str(opt_num_wc)].loc['Storage'])[:].rename(columns={str(opt_num_wc): 'S_wc'})[:decision_time * 4 + 1]

    d20 = pd.read_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    d20[str(opt_num_d20)].to_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_d20 = pd.DataFrame(d20[str(opt_num_d20)].loc['Storage'])[:].rename(columns={str(opt_num_d20): 'S_d20'})[:decision_time * 4 + 1]    
        
    esp = pd.read_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    esp[str(opt_num_esp)].to_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_esp = pd.DataFrame(esp[str(opt_num_esp)].loc['Storage'])[:].rename(columns={str(opt_num_esp): 'S_esp'})[:decision_time * 4 + 1]

    sffs = pd.read_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sffs[str(opt_num_sffs)].to_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_sffs = pd.DataFrame(sffs[str(opt_num_sffs)].loc['Storage'])[:].rename(columns={str(opt_num_sffs): 'S_sffs'})[:decision_time * 4 + 1]

    # Combine storage data from all scenarios into a single DataFrame and return
    S_all = pd.concat([S_pf, S_wc, S_d20, S_esp, S_sffs], axis=1).reset_index(drop=True)
    
    return S_all

In [168]:
# Main function for Reference point methods
def MCDM_RP(MCDM, year, month, reservoir_name, decision_time, leadtime):
    # Define file paths for different flow scenarios/forecasts
    path_pf = path + '/data/' + scenario_list[1] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Perfect forecast
    path_wc = path + '/data/' + scenario_list[2] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]   # Worst-case
    path_d20 = path + '/data/' + scenario_list[3] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # 20-year return period drought
    path_esp = path + '/data/' + scenario_list[4] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM]  # ESP (Ensemble Streamflow Prediction)
    path_sffs = path + '/data/' + scenario_list[5] + '/out/leadtime_' + str(leadtime) + '/' + MCDM_list[MCDM] # SFFs (Synthetic Flow Forecasts)

    # Read simulated Pareto front data for each flow scenario/forecast
    sim_pf = pd.read_csv(path_pf + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_wc = pd.read_csv(path_wc + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_d20 = pd.read_csv(path_d20 + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_esp = pd.read_csv(path_esp + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sim_sffs = pd.read_csv(path_sffs + '/[sim_pareto]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)

    # Normalization of the simulated Pareto front data
    scaler = MinMaxScaler()
    sim_all = pd.concat([sim_pf, sim_wc, sim_d20, sim_esp, sim_sffs], axis=0)  # Combine data from all scenarios
    nor_all = pd.DataFrame(scaler.fit(sim_all).transform(sim_all)).rename({0: 'SSD', 1: 'SVD'}, axis=1)  # Normalize data
    nor_all['number'] = nor_all.index  # Add index column for identification
    nor_all['remark'] = 'Perfect forecast'  # Default remark for Perfect forecast scenario
    
    for i in range(len(nor_all)):
        if (nor_all['number'][i] >= 100) and (nor_all['number'][i] < 200):
            nor_all['remark'][i] = 'Worst-case'
            nor_all['number'][i] = nor_all['number'][i] - 100
        elif (nor_all['number'][i] >= 200) and (nor_all['number'][i] < 300):
            nor_all['remark'][i] = '20-year drought'
            nor_all['number'][i] = nor_all['number'][i] - 200
        elif (nor_all['number'][i] >= 300) and (nor_all['number'][i] < 400):
            nor_all['remark'][i] = 'ESP'
            nor_all['number'][i] = nor_all['number'][i] - 300
        elif (nor_all['number'][i] >= 400) and (nor_all['number'][i] < 500):
            nor_all['remark'][i] = 'SFFs'
            nor_all['number'][i] = nor_all['number'][i] - 400

    nor_pf = nor_all[nor_all.remark == 'Perfect forecast']
    if MCDM == 6:  # Utopian point method
        nor_pf['distance'] = abs(nor_pf['SSD']**2 + nor_pf['SVD']**2)**0.5
        opt_num_pf = nor_pf[(nor_pf['distance'] == nor_pf['distance'].min())].index[0]
    elif MCDM == 7:  # Knee point method
        nor_pf['distance'] = abs(nor_pf['SSD'] + nor_pf['SVD'])
        opt_num_pf = nor_pf[(nor_pf['distance'] == nor_pf['distance'].min())].index[0]
    elif MCDM == 8:  # TOPSIS method
        nor_pf['distance+'] = abs(nor_pf['SSD']**2 + nor_pf['SVD']**2)**0.5
        nor_pf['distance-'] = abs((1-nor_pf['SSD'])**2 + (1-nor_pf['SVD'])**2)**0.5
        nor_pf['R_wc'] = nor_pf['distance-'] / (nor_pf['distance-']+nor_pf['distance+'])
        opt_num_pf = nor_pf[(nor_pf['R_wc'] == nor_pf['R_wc'].max())].index[0]
    else:
        drop
        
    nor_wc = nor_all[nor_all.remark == 'Worst-case']    
    if MCDM == 6:  # Utopian point method
        nor_wc['distance'] = abs(nor_wc['SSD']**2 + nor_wc['SVD']**2)**0.5
        opt_num_wc = nor_wc[(nor_wc['distance'] == nor_wc['distance'].min())].index[0]
    elif MCDM == 7:  # Knee point method
        nor_wc['distance'] = abs(nor_wc['SSD'] + nor_wc['SVD'])
        opt_num_wc = nor_wc[(nor_wc['distance'] == nor_wc['distance'].min())].index[0]
    elif MCDM == 8:  # TOPSIS method
        nor_wc['distance+'] = abs(nor_wc['SSD']**2 + nor_wc['SVD']**2)**0.5
        nor_wc['distance-'] = abs((1-nor_wc['SSD'])**2 + (1-nor_wc['SVD'])**2)**0.5
        nor_wc['R_wc'] = nor_wc['distance-'] / (nor_wc['distance-']+nor_wc['distance+'])
        opt_num_wc = nor_wc[(nor_wc['R_wc'] == nor_wc['R_wc'].max())].index[0]
    else:
        drop        

    nor_d20 = nor_all[nor_all.remark == '20-year drought']    
    if MCDM == 6:  # Utopian point method
        nor_d20['distance'] = abs(nor_d20['SSD']**2 + nor_d20['SVD']**2)**0.5
        opt_num_d20 = nor_d20[(nor_d20['distance'] == nor_d20['distance'].min())].index[0]
    elif MCDM == 7:  # Knee point method
        nor_d20['distance'] = abs(nor_d20['SSD'] + nor_d20['SVD'])
        opt_num_d20 = nor_d20[(nor_d20['distance'] == nor_d20['distance'].min())].index[0]
    elif MCDM == 8:  # TOPSIS method
        nor_d20['distance+'] = abs(nor_d20['SSD']**2 + nor_d20['SVD']**2)**0.5
        nor_d20['distance-'] = abs((1-nor_d20['SSD'])**2 + (1-nor_d20['SVD'])**2)**0.5
        nor_d20['R_d20'] = nor_d20['distance-'] / (nor_d20['distance-']+nor_d20['distance+'])
        opt_num_d20 = nor_d20[(nor_d20['R_d20'] == nor_d20['R_d20'].max())].index[0]
    else:
        drop        
        
    nor_esp = nor_all[nor_all.remark == 'ESP']
    if MCDM == 6:  # Utopian point method
        nor_esp['distance'] = abs(nor_esp['SSD']**2 + nor_esp['SVD']**2)**0.5
        opt_num_esp = nor_esp[(nor_esp['distance'] == nor_esp['distance'].min())].index[0]
    elif MCDM == 7:  # Knee point method
        nor_esp['distance'] = abs(nor_esp['SSD'] + nor_esp['SVD'])
        opt_num_esp = nor_esp[(nor_esp['distance'] == nor_esp['distance'].min())].index[0]
    elif MCDM == 8:  # TOPSIS method
        nor_esp['distance+'] = abs(nor_esp['SSD']**2 + nor_esp['SVD']**2)**0.5
        nor_esp['distance-'] = abs((1-nor_esp['SSD'])**2 + (1-nor_esp['SVD'])**2)**0.5
        nor_esp['R_esp'] = nor_esp['distance-'] / (nor_esp['distance-']+nor_esp['distance+'])
        opt_num_esp = nor_esp[(nor_esp['R_esp'] == nor_esp['R_esp'].max())].index[0]
    else:
        drop        

    nor_sffs = nor_all[nor_all.remark == 'SFFs']
    if MCDM == 6:  # Utopian point method
        nor_sffs['distance'] = abs(nor_sffs['SSD']**2 + nor_sffs['SVD']**2)**0.5
        opt_num_sffs = nor_sffs[(nor_sffs['distance'] == nor_sffs['distance'].min())].index[0]
    elif MCDM == 7:  # Knee point method
        nor_sffs['distance'] = abs(nor_sffs['SSD'] + nor_sffs['SVD'])
        opt_num_sffs = nor_sffs[(nor_sffs['distance'] == nor_sffs['distance'].min())].index[0]
    elif MCDM == 8:  # TOPSIS method
        nor_sffs['distance+'] = abs(nor_sffs['SSD']**2 + nor_sffs['SVD']**2)**0.5
        nor_sffs['distance-'] = abs((1-nor_sffs['SSD'])**2 + (1-nor_sffs['SVD'])**2)**0.5
        nor_sffs['R_sffs'] = nor_sffs['distance-'] / (nor_sffs['distance-']+nor_sffs['distance+'])
        opt_num_sffs = nor_sffs[(nor_sffs['R_sffs'] == nor_sffs['R_sffs'].max())].index[0]
    else:
        drop        
    
    # Load and save reservoir operation data for each scenario using the optimal numbers
    pf = pd.read_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    pf[str(opt_num_pf)].to_csv(path_pf + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_pf = pd.DataFrame(pf[str(opt_num_pf)].loc['Storage'])[:].rename(columns={str(opt_num_pf): 'S_pf'})[:decision_time * 4 + 1]

    wc = pd.read_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    wc[str(opt_num_wc)].to_csv(path_wc + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_wc = pd.DataFrame(wc[str(opt_num_wc)].loc['Storage'])[:].rename(columns={str(opt_num_wc): 'S_wc'})[:decision_time * 4 + 1]

    d20 = pd.read_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    d20[str(opt_num_d20)].to_csv(path_d20 + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_d20 = pd.DataFrame(d20[str(opt_num_d20)].loc['Storage'])[:].rename(columns={str(opt_num_d20): 'S_d20'})[:decision_time * 4 + 1]    
        
    esp = pd.read_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    esp[str(opt_num_esp)].to_csv(path_esp + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_esp = pd.DataFrame(esp[str(opt_num_esp)].loc['Storage'])[:].rename(columns={str(opt_num_esp): 'S_esp'})[:decision_time * 4 + 1]

    sffs = pd.read_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm.csv', index_col=0)
    sffs[str(opt_num_sffs)].to_csv(path_sffs + '/[Res_operation]' + str(reservoir_name) + '_' + str(year) + '_' + str(month) + '_decision_' + str(decision_time) + 'm_opt.csv')
    S_sffs = pd.DataFrame(sffs[str(opt_num_sffs)].loc['Storage'])[:].rename(columns={str(opt_num_sffs): 'S_sffs'})[:decision_time * 4 + 1]

    # Combine storage data from all scenarios and forecasts into a single DataFrame and return
    S_all = pd.concat([S_pf, S_wc, S_d20, S_esp, S_sffs], axis=1).reset_index(drop=True)
    
    return S_all

## 3. Continuous reservoir simulations

For the given simulation period, this code facilitates continuous release optimization by generating Pareto fronts and selecting a compromise release schedule using predefined functions. Please note that this process may require a significant amount of time, depending on the duration of the simulation, decision-making time step and number of iteration. For example, simulating over a period of 2.5 years can take approximately 6-7 hours.

### 3.1. Reservoir simulation settings

Now, it is necessary to specify the details of the reservoir simulations, including parameters such as storage names, capacity constraints, and release constraints.

In [169]:
# Define the flow scenario or forecast used in the analysis
scenario_list = {1:'1. Perfect forecast scenario', 2:'2. Worst case scenario', 3:'3. 20-year drought', 4:'4. ESP', 5:'5. SFFs'} 
# Define the MCDM method used in the analysis. Details of these MCDM methods will be explained in the next notebook.
MCDM_list = {1:'saw_bal', 2:'saw_sup', 3:'saw_sto', 4:'vw_select', 5:'vw_multi', 6:'rp_utopia', 7:'rp_knee', 8:'rp_topsis'}

# Define reservoir characteristic
reservoir_name = 'A'
s_min = 337               # Minimum Storage volume (Million Cubic Meters)
s_max = 4349              # Maximum Storage volume (Million Cubic Meters)
Q_max = 652               # Maximum regulated release (Million Cubic Meters/day)
env_min = 0               # Minimum environmental release (MCM/d)
Qreg_s_D_min = 0          # Minumum regulated release (MCM/d)
Qreg_s_D_max = Q_max      # Maximum regulated release (MCM/d)

# Details on the initial reservoir release optimisation
start_year = 2014
start_month = 6
end_year = 2016
end_month = 9

# Release optimisation options
pop = 100
itnum = 50000

# Weight allocation for SAW (Bal, Sup, Sto) and Simple Selective methods
weight_bal = 0.5    # Balanced weighting between supply and storage
weight_sup = 0.4    # Weight prioritizing supply (higher value favors supply)
weight_sto = 0.6    # Weight prioritizing storage (lower value favors storage)

### 3.2. Run the reservoir simulations

In [165]:
obsdata = pd.read_csv(path + '/data/observed_data.csv',index_col = 'Date')
ini_date = str(start_year) + '-' + str(start_month).zfill(2) + '-01'
s_ini = obsdata[obsdata.index == ini_date].iloc[0,0]    

# Continuous simulations
for leadtime in [2,4,6]:
    for decision_time in [1,2]:
        # Compute the iteration times for continuous simulation
        total_months = int(((end_year - start_year) * 12 + (end_month - start_month) + 1)/decision_time)
        
        for MCDM in range(4,5):  # for each MCDM method
            year = start_year
            month = start_month
            S_all = None

            for _ in range(total_months):     # for the number of iterations during defined simulation period
                # Optimize based on scenario
                for scenario in [1,2,3,4,5]:  # run the reservoir simulation functions for each flow forecast/scenario
                    
                    # Determine current S_value based on previous S_all for MCDM_VM method
                    if S_all is not None:  # if previous S_all is exist
                        if scenario == 1:
                            S_value = S_all['S_pf'].iloc[-1]
                        elif scenario == 2:
                            S_value = S_all['S_wc'].iloc[-1]
                        elif scenario == 3:
                            S_value = S_all['S_d20'].iloc[-1]
                        elif scenario == 4:
                            S_value = S_all['S_esp'].iloc[-1]
                        elif scenario == 5:
                            S_value = S_all['S_sffs'].iloc[-1]
                    else:  # if previous S_all is not exist
                        S_value = s_ini  # 초기값 또는 기본값 사용
                    
                    if scenario <= 3:
                        reservoir_opt_deterministic(reservoir_name, month, year, leadtime, pop, itnum, scenario) # Deterministic
                    else:
                        reservoir_opt_probabilistic(reservoir_name, month, year, leadtime, pop, itnum, scenario) # Probabilistic

                # When the Pareto front for release is generated, we select a single relese schedule using different MCDM methods
                if MCDM in [1, 2, 3]:    # Simple Additive Weighting (SAW) method
                    S_all = MCDM_SAW(MCDM, year, month, reservoir_name, decision_time, leadtime)
                elif MCDM in [4, 5]:     # Variable Weighting method
                    S_all = MCDM_VW(MCDM, year, month, reservoir_name, decision_time, leadtime)
                elif MCDM in [6, 7, 8]:  # Reference Point method
                    S_all = MCDM_RP(MCDM, year, month, reservoir_name, decision_time, leadtime)
                else:
                    continue  # Skip invalid MCDM values

                # Update month and year
                if month + decision_time > 12:
                    year += 1
                    month = month + decision_time - 12
                else:
                    month += decision_time
print("Successful")

Successful


In [170]:
# Load observed data from a CSV file, setting 'Date' as the index
obsdata = pd.read_csv(path + '/data/observed_data.csv', index_col='Date')
# Define the initial date based on the start year and month, and set the initial storage value
ini_date = str(start_year) + '-' + str(start_month).zfill(2) + '-01'
s_ini = obsdata[obsdata.index == ini_date].iloc[0, 0]    

# Start continuous simulations
for leadtime in [2, 4, 6]:  # Different lead times for simulation
    for decision_time in [1, 2]:  # Different decision intervals
        # Calculate the total number of months for the continuous simulation
        total_months = int(((end_year - start_year) * 12 + (end_month - start_month) + 1) / decision_time)
        
        for MCDM in range(5, 6):  # Loop through different MCDM methods (currently set to a single method)
            year = start_year  # Initialize the simulation year
            month = start_month  # Initialize the simulation month
            S_all = None  # Initialize S_all to store results from each simulation round

            # Iterate through each month in the simulation period
            for _ in range(total_months):
                # Optimize based on scenario
                for scenario in [1, 2, 3, 4, 5]:  # Loop through different scenarios for reservoir management
                    
                    # Determine the current S_value based on the previous S_all for the Variable Weighting method
                    if S_all is not None:  # If previous S_all exists
                        if scenario == 1:
                            S_value = S_all['S_pf'].iloc[-1]  # Get value from Pareto front for scenario 1
                        elif scenario == 2:
                            S_value = S_all['S_wc'].iloc[-1]  # Get value for water conservation
                        elif scenario == 3:
                            S_value = S_all['S_d20'].iloc[-1]  # Get value for 20-day drought scenario
                        elif scenario == 4:
                            S_value = S_all['S_esp'].iloc[-1]  # Get value from ESP method
                        elif scenario == 5:
                            S_value = S_all['S_sffs'].iloc[-1]  # Get value from SFFS method
                    else:  # If previous S_all does not exist
                        S_value = s_ini  # Use initial value from observed data as a fallback
                    
                    # Optimize reservoir operations based on scenario
                    if scenario <= 3:
                        reservoir_opt_deterministic(reservoir_name, month, year, leadtime, pop, itnum, scenario)  # Use deterministic optimization
                    else:
                        reservoir_opt_probabilistic(reservoir_name, month, year, leadtime, pop, itnum, scenario)  # Use probabilistic optimization

                # After generating the Pareto front for releases, select a single release schedule using different MCDM methods
                if MCDM in [1, 2, 3]:  # Simple Additive Weighting (SAW) method
                    S_all = MCDM_SAW(MCDM, year, month, reservoir_name, decision_time, leadtime)
                elif MCDM in [4, 5]:  # Variable Weighting method
                    S_all = MCDM_VW(MCDM, year, month, reservoir_name, decision_time, leadtime, scenario)
                elif MCDM in [6, 7, 8]:  # Reference Point method
                    S_all = MCDM_RP(MCDM, year, month, reservoir_name, decision_time, leadtime)
                else:
                    continue  # Skip any invalid MCDM values

                # Update month and year for the next iteration
                if month + decision_time > 12:
                    year += 1  # Move to the next year if the month exceeds 12
                    month = month + decision_time - 12  # Calculate the new month
                else:
                    month += decision_time  # Otherwise, just increment the month
            
print("Successful")  # Indicate that the simulation has completed successfully

Successful
